In [1]:
import dash
import dash_table
import pandas as pd

In [11]:
df = pd.read_csv('2020-02-05-02-00_data.csv')
df.head()

,Province/State,Country/Region,Last Update,Confirmed,Recovered,Deaths,City,Date_last_updated_AEDT,lat,lon
0,Hubei,China,02/05/2020 02:00,16678,522,479,NaN,2020-02-05 18:00:00,31.151725,112.878322
1,Zhejiang,China,02/05/2020 02:00,895,72,0,NaN,2020-02-05 18:00:00,29.000000,120.000000
2,Guangdong,China,02/05/2020 02:00,870,33,0,NaN,2020-02-05 18:00:00,23.135769,113.198269
3,Henan,China,02/05/2020 02:00,764,47,2,NaN,2020-02-05 18:00:00,34.000000,114.000000
4,Hunan,China,02/05/2020 02:00,661,37,0,NaN,2020-02-05 18:00:00,27.666209,111.748706


In [12]:
df = df[['Province/State', 'Country/Region','Confirmed']]
df

,Province/State,Country/Region,Confirmed
0,Hubei,China,16678
1,Zhejiang,China,895
2,Guangdong,China,870
3,Henan,China,764
4,Hunan,China,661
...,...,...,...
61,NaN,Sweden,1
62,Massachusetts,US,1
63,WA,US,1
64,Arizona,US,1


In [13]:
dfSum = df.groupby(by='Country/Region', sort=False).sum().reset_index()
dfSum = dfSum.sort_values(by=['Confirmed'], ascending=False).reset_index(drop=True)

In [14]:
dfSum

,Country/Region,Confirmed
0,China,24339
1,Japan,33
2,Thailand,25
3,Singapore,24
4,Hong Kong,18
5,South Korea,18
6,Australia,13
7,Germany,12
8,Taiwan,11
9,US,11


In [89]:
app = dash.Dash(__name__)

app.layout = dash_table.DataTable(
    id='datatable-interactivity',
    columns=[{"name": i, "id": i} for i in dfSum.columns],
    data=dfSum.to_dict("rows"),
    row_selectable="single",
    selected_rows=[],
    sort_action="native",
    style_as_list_view=True,
    style_cell={'padding': '1px'},
    style_header={
        'backgroundColor': 'white',
        'fontWeight': 'bold'
    }
)
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [84]:
import dash
from dash.dependencies import Input, Output
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

app = dash.Dash(__name__)

app.layout = html.Div([
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Div(id='datatable-interactivity-container')
])

@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('datatable-interactivity-container', "children"),
    [Input('datatable-interactivity', "derived_virtual_data"),
     Input('datatable-interactivity', "derived_virtual_selected_rows")])
def update_graphs(rows, derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncracy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)

    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]

    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [
                    {
                        "x": dff["country"],
                        "y": dff[column],
                        "type": "bar",
                        "marker": {"color": colors},
                    }
                ],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {
                        "automargin": True,
                        "title": {"text": column}
                    },
                    "height": 250,
                    "margin": {"t": 10, "l": 10, "r": 10},
                },
            },
        )
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        for column in ["pop", "lifeExp", "gdpPercap"] if column in dff
    ]


if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_arg

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/anaconda3/lib/python3.6/site-packages/dash/dash.py", line 1371, in dispatch
    response.set_data(self.callback_map[output

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/anaconda3/lib/python3.6/site-packages/dash/dash.py", line 1371, in dispatch
    response.set_data(self.callback_map[output